# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [108]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [109]:
data = "output_data/cities.csv"
cities_df = pd.DataFrame(pd.read_csv(data))
cities_df.head()

humidity = cities_df["Humidity"].astype(float)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [110]:
target_coordinates = cities_df[["Lat", "Lon"]]
target_coordinates

#humidity = cities_df["Humidity"]

fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(target_coordinates, weights=humidity, dissipating=True, max_intensity=100) 
                                  

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig



# # Build the endpoint URL
# target_url = ('https://maps.googleapis.com/maps/api/geocode/json?'
#     'address={0}&key={1}').format(target_city, gkey)
# geo_data = requests.get(target_url).json()

Figure(layout=FigureLayout(height='420px'))

Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [111]:
hotel_df = pd.DataFrame(cities_df.loc[(cities_df['Max Temp'] < 80) & 
                             (cities_df['Max Temp'] > 70) &
                             (cities_df['Wind Speed'] < 9) &
                             (cities_df['Cloudiness'] == 0)])
myweather_df

,City_ID,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
11,11,Luau,-10.71,22.22,77.79,29,0,1.30,AO,1593360234
160,160,Caravelas,-17.71,-39.25,79.66,63,0,7.94,BR,1593360252
186,186,Guerrero Negro,27.98,-114.06,71.29,65,0,5.39,MX,1593360227
232,232,Shitanjing,39.23,106.34,72.00,37,0,4.00,CN,1593360260
233,233,Rassvet,46.83,40.75,78.64,46,0,5.21,RU,1593360260
274,274,Lyuban',52.80,28.00,72.52,78,0,8.90,BY,1593360265
337,337,Kanelovskaya,46.59,39.19,79.20,50,0,4.72,RU,1593360272
379,379,Acarí,-15.43,-74.62,76.03,37,0,7.36,PE,1593360276
413,413,Ambilobe,-13.20,49.05,74.37,81,0,0.51,MG,1593360281
435,435,Yangi Marg`ilon,40.43,71.72,75.20,22,0,6.93,UZ,1593360283


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [112]:
hotel_df["Hotel Name"] = ""
hotel_df

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "lodging",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lon = row["Lon"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lon}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    hotel_name = requests.get(base_url, params=params).json()
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
        
    except (KeyError, IndexError):
         print("Missing field/result... skipping.")


Missing field/result... skipping.
Missing field/result... skipping.


In [113]:
hotel_df

,City_ID,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
92,92,Yuxia,34.06,108.63,77.00,88,0,4.47,CN,1593470687,Xi'an drunken folk house
203,203,Verkhnevilyuysk,63.45,120.32,74.75,49,0,2.57,RU,1593470699,Gostinitsa Mila
247,247,Chaman,30.92,66.45,76.69,20,0,4.03,PK,1593470645,محلہ حاجی صلاح الدین عشیزئی چمن
261,261,Saravan,39.72,45.65,73.40,35,0,6.93,AM,1593470706,
299,299,Rodopoli,41.26,23.00,72.00,73,0,1.79,GR,1593470710,Αρχοντικό Βιγλάτορας (Viglatoras Traditional Inn)
317,317,Fethiye,36.62,29.12,71.60,88,0,4.09,TR,1593470713,Misafir Evi Hotel
334,334,Alghero,40.56,8.32,73.40,94,0,3.36,IT,1593470715,Hotel Florida
376,376,Mokhsogollokh,61.47,128.92,75.40,51,0,5.35,RU,1593470719,
412,412,Alta Floresta,-9.88,-56.09,73.96,53,0,3.20,BR,1593470483,Hotel Mandino
547,547,Barcelos,41.54,-8.62,71.01,82,0,3.36,PT,1593470740,Quinta de Santa Comba


In [114]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lon"]]

In [115]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
# Add the layer to the map
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))